In [85]:
%matplotlib ipympl

import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.linalg
import msmtools.estimation

In [86]:
import sys
sys.path.append('..')
from bkit.milestoning import MilestoningModel

In [87]:
trajs = []
traj_ids = []

p = '/data/p38a-SB2/short_md/{}/x.joblib'
for v in ['bound', 'path1']:
    fn = p.format(v)
    data = joblib.load(fn)
    for traj_id, traj in data.items():
        trajs.append(traj[:, np.newaxis])
        traj_ids.append((v, traj_id[0], traj_id[1]))        

In [88]:
plt.figure()

n, bins, _ = plt.hist(np.concatenate(trajs), 1000, density=True, histtype='step')
plt.ylabel('Empirical Density')
_ = plt.xlabel('Reaction Coordinate ($\mathrm{\AA}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
anchors = np.arange(14.0, 29.0, 1.0)[:, np.newaxis]

In [90]:
model = MilestoningModel(anchors)
model.load_trajectory_data(trajs, dt=0.1)

In [91]:
print(model.is_resolved)

True


In [92]:
model.estimate()

In [93]:
print(model.K)

[[0.         1.         0.         ... 0.         0.         0.        ]
 [0.06470062 0.         0.93529938 ... 0.         0.         0.        ]
 [0.         0.67526403 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.88647746 0.        ]
 [0.         0.         0.         ... 0.2987239  0.         0.7012761 ]
 [0.         0.         0.         ... 0.         1.         0.        ]]


In [99]:
K = msmtools.estimation.transition_matrix(model._N, reversible=True)

In [109]:
K[1,:]

array([6.47006167e-02, 2.22044605e-16, 9.35299383e-01, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [47]:
print('Mean lifetimes:')
for milestone, t, err in zip(model.milestones, model.t, model.t_stderr):
    print('{:8} : {:4.1f} ps +/- {:.1f} ps'.format(str(milestone), t, err))

Mean lifetimes:
{0, 1}   :  5.6 ps +/- 0.2 ps
{1, 2}   :  8.0 ps +/- 0.1 ps
{2, 3}   : 11.8 ps +/- 0.1 ps
{3, 4}   :  6.7 ps +/- 0.1 ps
{4, 5}   :  5.9 ps +/- 0.2 ps
{5, 6}   :  7.3 ps +/- 0.2 ps
{6, 7}   :  8.0 ps +/- 0.2 ps
{8, 7}   :  7.5 ps +/- 0.2 ps
{8, 9}   :  6.8 ps +/- 0.2 ps
{9, 10}  :  7.4 ps +/- 0.3 ps
{10, 11} :  5.9 ps +/- 0.4 ps
{11, 12} :  5.3 ps +/- 0.3 ps
{12, 13} :  6.9 ps +/- 0.2 ps
{13, 14} :  7.2 ps +/- 0.3 ps


In [48]:
plt.close(plt.gcf())

fig, ax = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True)

for k in range(12):
    ax_ = ax[k // 4, k % 4]
    ax_.hist(model._lifetimes[k], bins=20, histtype='step', density=True)

    tmax = 40
    s = np.arange(0, tmax, 0.1)
    ax_.plot(s, np.exp(-s / model.t[k]) / model.t[k])

    ax_.set_xlim((0, tmax))
    
    ax_.text(tmax / 2, 0.16, model.milestones[k], horizontalalignment='center')
    
    if not k % 4:
        ax_.set_ylabel('P(t)')
    
    if k // 4 >= 2:
        ax_.set_xlabel('Lifetime (ps)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
eigvals, eigvecs = scipy.linalg.eig(model.K, left=True, right=False)
q = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q = -q
print(q)

[0.03579015 0.5531655  0.76618231 ... 0.01928153 0.05721885 0.04012621]


In [117]:
for i in range(len(K) - 1):
    j1 = q[i] * model.K[i, i+1]
    j2 = q[i+1] * model.K[i+1, i]
    print('{} - {} = {}'.format(j1, j2, j1 - j2))

0.035790148869938976 - 0.03579014886993902 = -4.163336342344337e-17
0.5173753495748986 - 0.5173753495748984 = 2.220446049250313e-16
0.2488069560675115 - 0.24880695606751113 = 3.608224830031759e-16
0.027235613280278195 - 0.027235613280277838 = 3.5735303605122226e-16
0.015599085884913228 - 0.015599085884912652 = 5.759281940243e-16
0.04079119250118869 - 0.04079119250118785 = 8.396061623727746e-16
0.051277997051088375 - 0.051277997051087404 = 9.71445146547012e-16
0.03461198136885671 - 0.03461198136885585 = 8.604228440844963e-16
0.017279689786731226 - 0.017279689786730387 = 8.396061623727746e-16
0.007016434547159632 - 0.007016434547158803 = 8.291978215169138e-16
0.00218888770060937 - 0.0021888877006086744 = 6.956241138666996e-16
0.01709263777975303 - 0.01709263777975256 = 4.718447854656915e-16
0.0401262117975162 - 0.04012621179751618 = 2.0816681711721685e-17


In [116]:
q[2] * model.K[2,1]

0.5173753495748984

In [113]:
eigvals, eigvecs = scipy.linalg.eig(K, left=True, right=False)
q = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q = -q
print(q)

[0.03579015 0.5531655  0.76618231 ... 0.01928153 0.05721885 0.04012621]


In [118]:
p = q * model.t

In [120]:
Q = np.zeros(K.shape)
for a in range(len(Q)):
    for b in range(len(Q)):
        if b == a:
            Q[a, a] = -1 / model.t[a]
        else:
            Q[a, b] = model.K[a, b] / model.t[a]

In [50]:
plt.close(plt.gcf())
plt.figure()

RT = 0.593 # kcal/mol

err = RT * model.t_stderr / model.t
p = q * model.t
p = p / sum(p)
F = -RT * np.log(p)

plt.errorbar(range(len(q)), F, yerr=err)
plt.ylabel('Free energy (kcal/mol)')
_ = plt.xlabel('Milestone index')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
p_milestone = [0] + list(p) + [0]
p_cell = [(p_milestone[i] + p_milestone[i+1]) / 2 for i in range(len(p_milestone)-1)]

In [52]:
plt.figure()

widths = [0.5] + [1.] * (len(p_cell) - 2) + [0.5]
p = np.divide(p_cell, widths)
plt.step(anchors.flatten(), -RT * np.log(p), where='mid')
plt.plot(anchors.flatten(), -RT * np.log(p), 'o--', color='grey', alpha=0.5)
plt.xlim([14, 28])
plt.ylabel('Free energy (kcal/mol)')
_ = plt.xlabel('Reaction coordinate ($\mathrm{\AA}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

widths = [0.5] + [1.] * (len(p_cell) - 2) + [0.5]
p = np.divide(p_cell, widths)
F = -RT * np.log(p)

x = [np.mean([bins[i-1], bins[i]]) for i in range(1, len(bins))]
F_emp = -RT * np.log(n)

F = F - min(F) + min(F_emp)

ax[0].step(anchors.flatten(), F, where='mid')
ax[0].plot(anchors.flatten(), F, 'o--', color='grey', alpha=0.5)
ax[0].set_xlim([14, 28])
ax[0].set_ylim([0, 4])
ax[0].set_ylabel('Milestoning')

ax[1].plot(x, F_emp)
ax[1].set_ylabel('Empirical')
ax[1].set_xlabel('Reaction coordinate ($\mathrm{\AA}$)')
ax[1].set_ylim([0, 4])

In [ ]:
reactant, product = 2, 13

# Apply cyclic boundary conditions
model.K[product, :] = 0
model.K[product, reactant] = 1

print(model.K)

In [ ]:
eigvals, eigvecs = scipy.linalg.eig(model.K, left=True, right=False)
q_cyc = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
if all(q < 0):
    q_cyc = -q_cyc
print(q_cyc)

In [ ]:
mfpt = np.dot(q_cyc, model.t) / q_cyc[product]
s = 'MFPT from milestone {} to milestone {}: {:.2f} ns'
print(s.format(model.milestones[reactant], model.milestones[product], mfpt / 1000))

Extract first hitting frames for a given milestone:

In [76]:
milestone = {9, 10}

first_hitting_frames = {traj_id: [] for traj_id in traj_ids}

for traj_id, dtraj in zip(traj_ids, model._dtrajs):
    in_milestone = False
    for frame_index, (i, j) in enumerate(zip(dtraj[:-1], dtraj[1:])):
        if j == i:
            continue
        
        if {i, j} != milestone:
            in_milestone = False
            continue
            
        if not in_milestone:
            first_hitting_frames[traj_id].append(frame_index)
        in_milestone = True

In [77]:
nframes = sum(map(len, first_hitting_frames.values()))
print(nframes)

956


In [78]:
import pytraj as pt

In [79]:
tn = '/data/p38a-SB2/complex.prmtop'
traj = pt.Trajectory(top=tn)
for traj_id, frame_indices in first_hitting_frames.items():
    if not frame_indices:
        continue
    fn = '/data/p38a-SB2/short_md/{}/c{}/r{}/mdcrd'.format(*traj_id)
    traj.append(pt.load(fn, tn, frame_indices=frame_indices))

In [80]:
traj

pytraj.Trajectory, 956 frames: 
Size: 0.121286 (GB)
<Topology: 5676 atoms, 350 residues, 2 mols, PBC with box type = ortho>
           

In [84]:
x = '%g' % (sum([anchors[i] for i in milestone]) / 2)
fn = '/home/jefft/milestone_{}.dcd'.format(x)
pt.save(fn, traj)
print('Saved first hitting frames as ' + fn)

Saved /home/jefft/milestone_23.5.dcd
